In [55]:
import numpy as np
from scipy.linalg import polar
def printMat(M):
    for i in range(3):
        print(f"{M[i,0]:.4f} & {M[i,1]:.4f} & {M[i,2]:.4f} \\\ ")  

In [56]:
DATA = [
    [[0.63, 0.45, 0.87], [0.17, 0.03, 0.50]],
    [[0.03, 0.40, 0.86], [0.26, 0.37, 0.15]],
    [[0.86, 0.17, 0.16], [0.78, 0.65, 0.67]],
    [[0.29, 0.79, 0.04], [0.83, 0.96, 0.82]],
    [[0.60, 0.38, 0.15], [0.69, 0.72, 0.04]],
    [[0.99, 0.43, 0.76], [0.64, 0.77, 0.34]]
]

for i in range(len(DATA)) :
    DATA[i][0] = np.array(DATA[i][0]) / np.linalg.norm(np.array(DATA[i][0]) )
    DATA[i][1] = np.array(DATA[i][1]) / np.linalg.norm(np.array(DATA[i][1]) )
    
def MSE(A):
    mse = 0
    for i in range(len(DATA)) :
        v = DATA[i][0]
        w = DATA[i][1] 
        mse += np.linalg.norm(A @ v - w)**2
    return mse / len(DATA)


#     mse += np.linalg.norm(DATA[i][0] - DATA[i][1])**2
# print(mse/len(DATA))
print(MSE(np.eye(3)))

0.3347562818077981


In [57]:
B = np.zeros((3,3))
for i in range(len(DATA)):
    w = DATA[i][1]
    v = DATA[i][0]
    w = w.reshape((len(w),1))
    v = v.reshape((len(v),1))
    B += w @ v.T
U, S , Vt = np.linalg.svd(B)
M = np.diag([1, 1, np.linalg.det(U) * np.linalg.det(Vt)])
R = (U @ M @ Vt)

printMat(R)

print(MSE(R))

0.9962 & -0.0324 & -0.0812 \\ 
0.0401 & 0.9947 & 0.0952 \\ 
0.0777 & -0.0981 & 0.9921 \\ 
0.3272077298366734


In [58]:
### TRIAD ###
def TRAID(v1, w1, v2 ,w2) :
    r1 = v1 / np.linalg.norm(v1)
    r2 = np.cross(v1, v2) 
    r2 = r2 / np.linalg.norm(r2)
    r3 = np.cross(r1, r2)

    s1 = w1 / np.linalg.norm(w1)
    s2 = np.cross(w1, w2) 
    s2 = s2 / np.linalg.norm(s2)
    s3 = np.cross(s1, s2)

    s1 = s1.reshape((-1, 1))
    s2 = s2.reshape((-1, 1))
    s3 = s3.reshape((-1, 1))

    r1 = r1.reshape((-1, 1))
    r2 = r2.reshape((-1, 1))
    r3 = r3.reshape((-1, 1))

    Mobs = np.hstack((s1, s2, s3))
    Mref = np.hstack((r1, r2, r3))
    A = Mobs @ Mref.T 
    return A

In [59]:
v1 = np.array(DATA[0][0])
w1 = np.array(DATA[0][1])
v2 = np.array(DATA[1][0])
w2 = np.array(DATA[1][1])

A1 = TRAID(v1, w1, v2, w2)
A2 = TRAID(v2, w2, v1, w1)

print(MSE(A1))
print(MSE(A2))
printMat(A2)

1.161573692516731
0.6676595782614001
0.0057 & 0.9899 & 0.1415 \\ 
-0.3439 & -0.1310 & 0.9298 \\ 
0.9390 & -0.0539 & 0.3397 \\ 


In [60]:
### QUEST ###
sigma = np.trace(B)
S = (B + B.T)
Z = np.zeros(3)
for i in range(len(DATA)) :
    w = DATA[i][1]
    v = DATA[i][0]
    Z = Z + np.cross(w , v)
K = np.zeros((4,4))
K[:3, :3] = S - sigma * np.eye(3)
K[3,3] = sigma
K[3,:3] = Z
K[:3,3] = Z

q = np.linalg.eig(K)[1][:,3]

print(np.linalg.eig(K)[1])
# q = np.array([1, 0, 0, 0])
# eigs = []
# new_eig = 0
# while True :
#     old_eig = new_eig
#     q = K @ q
#     q = q / np.linalg.norm(q)
#     new_eig = q.T @ K @ q
#     if abs(new_eig - old_eig) < 1e-6 :
#         break
#     eigs.append(new_eig)

# import matplotlib.pyplot as plt
# print(eigs[-1])
# plt.plot(eigs)
# plt.show()
# print(np.linalg.eigvals(K))

[[ 0.53182145 -0.55247916 -0.63999087  0.04841901]
 [-0.80653839 -0.09939355 -0.58140467  0.0398165 ]
 [ 0.2579518   0.82631003 -0.50034234 -0.01816974]
 [ 0.0110738   0.04581954  0.04514239  0.99786778]]


In [61]:
def sk(q) :
    return np.array([
        [0, q[2], -q[1]],
        [-q[2], 0, q[0]],
        [q[1], -q[0], 0]
    ])

Q = q[:3]
q = q[3]
A = np.eye(3) * (q ** 2 - Q.T @ Q) + 2 * Q @ Q.T + 2 * q * sk(Q)
print(MSE(A))

0.3252090424296568


In [62]:
  
printMat(A)        

1.0000 & -0.0277 & -0.0709 \\ 
0.0448 & 1.0000 & 0.1052 \\ 
0.0880 & -0.0881 & 1.0000 \\ 
